In [24]:
# En quête de fripes  
## Analyse des flux et impacts des vêtements usagés

# Projet réalisé dans le cadre du bootcamp Data Analytics du Wagon.

In [25]:
## Contexte
# Ce notebook vise à documenter la démarche d’analyse de données ayant conduit à la présentation finale du projet « En quête de fripes ».

# L’objectif est de rendre l’analyse compréhensible.

In [26]:
## Objectifs de l’analyse
# - Identifier des indicateurs clés
# - Produire des visualisations claires
# - Interpréter les résultats*/

In [28]:
## Données
# Les données utilisées proviennent de sources ouvertes mentionnées dans la présentation finale du projet.

# Elles sont chargées et analysées ci-dessous.

In [29]:
## Données
# Les données utilisées proviennent de sources ouvertes mentionnées dans le document de cadrage.

# Elles sont chargées et analysées ci-dessous.

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
## Première partie de l'analyse : imports exports des vêtements usagés.
# Source : https://comtradeplus.un.org (ONU echanges internationaux)
# Nous avons un fichier par année, pour réaliser notre analyse sur 10 ans (de 2014 à 2024) il faut joindre tout les csv téléchargés en une seule table.

folder = ['trade_data/TradeData_2014.csv', 'trade_data/TradeData_2015.csv', 'trade_data/TradeData_2016.csv', 'trade_data/TradeData_2017.csv', 'trade_data/TradeData_2018.csv', 'trade_data/TradeData_2019.csv', 'trade_data/TradeData_2020.csv', 'trade_data/TradeData_2021.csv', 'trade_data/TradeData_2022.csv', 'trade_data/TradeData_2023.csv', 'trade_data/TradeData_2024.csv']

df_list = []
for file in folder :
    df_list.append(pd.read_csv(file, encoding="latin1"))
data = pd.concat(df_list, ignore_index=True)

#On obtient une table de 47 colonnes et 24832 lignes
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24832 entries, 0 to 24831
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   typeCode                  24832 non-null  object 
 1   freqCode                  24832 non-null  int64  
 2   refPeriodId               24832 non-null  int64  
 3   refYear                   24832 non-null  int64  
 4   refMonth                  24832 non-null  int64  
 5   period                    24832 non-null  int64  
 6   reporterCode              24832 non-null  object 
 7   reporterISO               24832 non-null  object 
 8   reporterDesc              24832 non-null  object 
 9   flowCode                  24832 non-null  object 
 10  flowDesc                  24832 non-null  int64  
 11  partnerCode               24832 non-null  object 
 12  partnerISO                24832 non-null  object 
 13  partnerDesc               24832 non-null  int64  
 14  partne

In [31]:
data.head()

,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,...,netWgt,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate
0,M,20240101,2024,1,202401,251,FRA,France,M,Import,...,True,0,False,1782446.709,NaN,1782446.709,6,False,True,NaN
1,M,20240101,2024,1,202401,251,FRA,France,M,Import,...,True,0,False,30.534,NaN,30.534,6,False,True,NaN
2,M,20240101,2024,1,202401,251,FRA,France,M,Import,...,False,0,False,4.362,NaN,4.362,0,False,True,NaN
3,M,20240101,2024,1,202401,251,FRA,France,M,Import,...,True,0,False,750.273,NaN,750.273,6,False,True,NaN
4,M,20240101,2024,1,202401,251,FRA,France,M,Import,...,False,0,False,430.753,NaN,430.753,0,False,True,NaN


In [32]:
# Nous devons maintenant séléctionner les colonnes qui nous interessent pour notre étude.
df_selected_data = data[['refPeriodId', 'refMonth', 'flowCode', 'partnerISO', 'isAltQtyEstimated', 'fobvalue']]
df_selected_data.head(10)


,refPeriodId,refMonth,flowCode,partnerISO,isAltQtyEstimated,fobvalue
0,2024,202401,Import,World,910346.167,1782446.709
1,2024,202401,Import,Afghanistan,15.595,30.534
2,2024,202401,Import,Algeria,5.000,4.362
3,2024,202401,Import,Australia,88.303,750.273
4,2024,202401,Import,Brazil,113.000,430.753
5,2024,202401,Import,Myanmar,17.266,33.806
6,2024,202401,Import,Canada,126.872,343.512
7,2024,202401,Import,Chile,120.860,236.641
8,2024,202401,Import,China,25457.546,96366.508
9,2024,202401,Import,Dominican Rep.,8741.000,6269.363


In [33]:
# Nous devons ensuite renomer les colonnes pour plus de clarté

df_selected_data = df_selected_data.rename(columns={'refPeriodId': "annee", 'refMonth': "date_date", 'flowCode': "flux", 'partnerISO': "partenaire", 'isAltQtyEstimated': "poids_kg", 'fobvalue': "valeux_USD"})
df_selected_data.head()

,annee,date_date,flux,partenaire,poids_kg,valeux_USD
0,2024,202401,Import,World,910346.167,1782446.709
1,2024,202401,Import,Afghanistan,15.595,30.534
2,2024,202401,Import,Algeria,5.000,4.362
3,2024,202401,Import,Australia,88.303,750.273
4,2024,202401,Import,Brazil,113.000,430.753


In [34]:
# Nous pouvons ensuite exporter notre dataframe en csv

df_selected_data.to_csv('import_export_france.csv', index=False)